In [1]:

import numpy as np 
import math
import pandas as pd
import warnings
from collections import Counter
import random

train_file='../Dataset/breast-cancer-dataset/train.csv'
test_file='../Dataset/breast-cancer-dataset/test.csv'
validate_file='../Dataset/breast-cancer-dataset/val.csv'

In [2]:
train_data=pd.read_csv(train_file)
test_data=pd.read_csv(test_file, header=None)
validate_data=pd.read_csv(validate_file, header=None)

train_data.drop(['id'],1, inplace=True)
test_data.drop([0],1, inplace=True)
validate_data.drop([0],1, inplace=True)

/var/folders/l4/s0n110fx5qjfmsh_w_yxrcqw0000gp/T/ipykernel_2327/3970750545.py:5: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  train_data.drop(['id'],1, inplace=True)
/var/folders/l4/s0n110fx5qjfmsh_w_yxrcqw0000gp/T/ipykernel_2327/3970750545.py:6: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  test_data.drop([0],1, inplace=True)
/var/folders/l4/s0n110fx5qjfmsh_w_yxrcqw0000gp/T/ipykernel_2327/3970750545.py:7: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  validate_data.drop([0],1, inplace=True)


In [3]:
train_data=train_data.astype(float).values.tolist()
test_data=test_data.astype(float).values.tolist()
validate_data=validate_data.astype(float).values.tolist()

In [4]:
# dictionaries have to keys: 2 for benign tumors and 4 for malignant
train_set={2:[],4:[]}  
test_set={2:[],4:[]}
validate_set={2:[],4:[]}

for i in train_data:
    train_set[i[-1]].append(i[:-1]) # appending list to last element
for i in test_data:
    test_set[i[-1]].append(i[:-1])
for i in validate_data:
    validate_set[i[-1]].append(i[:-1])

In [5]:
from pprint import pprint

In [11]:
#data to train, new data to predict and a value for k
def k_nearest_neighbors(data,predict,k):
    distances=[]  # list of distances
    for group in data: # each class in data
        for features in data[group]: # iterating through features
            euclidean_distance = math.sqrt( (features[0]-predict[0])**2 + (features[1]-predict[1])**2 )
            distances.append([euclidean_distance,group])  
    votes=[i[1] for i in sorted(distances)[:k]]  
    vote_result=Counter(votes).most_common(1)[0][0] 
    return vote_result

In [13]:
print(len(train_data))

477


In [14]:
train_size=len(train_data)

In [15]:
# test with different odd k values since binary classification
k_range=range(1,train_size)
correct=0
total=0
max_accuracy=0.0
optimal_k=0
accuracy=0.0
accuracies=[]
for k in k_range:
    # iterate through groups in testset
    for group in validate_set:
        #go through data point , test on validate_set
        for data in validate_set[group]:
            # feed data point into function 
            vote=k_nearest_neighbors(train_set, data, k)
            # if group from test_set equal to vote from knn classifier
            if group==vote:
                correct+=1
            total+=1 
            accuracy=correct/total      
    if max_accuracy<accuracy:
            max_accuracy=accuracy
            optimal_k=k   
    accuracies.append(accuracy)

In [16]:
# median of k ?? 
print('max accuracy: ', max_accuracy)
print('optimal k: ',optimal_k)

max accuracy:  0.9907770515613653
optimal k:  102


In [17]:
# test with test_set with optimal k and find accuracy
for group in test_set:
        #go through data point , test on validate_set
    for data in test_set[group]:
            # feed data point into function 
        vote=k_nearest_neighbors(train_set, data, optimal_k)
        if group==vote:
            correct+=1
        total+=1
        accuracy=correct/total
print('Accuracy: ', accuracy)

Accuracy:  0.8715313942823387
